In [13]:
import keras
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import time

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector

In [17]:
import util.broadcast_vector as BV
import util.tpm_from_vcs as get_TPM

In [18]:
np.random.seed(23)
MAX_NODES = 100 # Atmost 10 points in the network for now
MAX_VC_DIM = 10
HIDDEN_SIZE = 1024

In [19]:
def create_dataset(size):
    X = [] # array of tuples
    y = []
    
    
    for i in range(0, size):
        # effectively 100 numbers in one-timestep is fed into rnn (m distances)
        m = np.random.randint(low=3, high=10)
        # number of anchors could be 1% of total nodes
        n = np.random.randint(low=m, high=100) 
        P = np.random.rand(n, m)

        ans = get_TPM.generate_tpm_from_vcs(P)

        if np.iscomplex(ans.flatten()).any():
            print("Is the TC matrix complex : {}".format(np.iscomplex(ans.flatten())))
        
        # broadcast the input vector into MAX_VC_DIM length
        P = BV.get_vector(MAX_VC_DIM, P)
        ans = BV.get_vector(MAX_VC_DIM, ans)

        # dataset[i] = (P.tolist(), ans.tolist())
#         dataset.append((P, ans))
        X.append(P)
        y.append(ans)
    
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

In [8]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAX_NODES, MAX_VC_DIM)))
model.add(RepeatVector(MAX_NODES))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
model.add(Dense(MAX_VC_DIM))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1024)              4239360   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 100, 1024)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 1024)         8392704   
_________________________________________________________________
dense_2 (Dense)              (None, 100, 10)           10250     
Total params: 12,642,314
Trainable params: 12,642,314
Non-trainable params: 0
_________________________________________________________________


In [9]:
inp_data = np.random.randn(2, 100, 10)

In [10]:
out_data = np.random.randn(2, 100, 10)

In [11]:
model.fit(inp_data, out_data, epochs=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
2/2 [==============================] - 5s 3s/step - loss: 1.0002 - acc: 0.1000


In [21]:
X, y = create_dataset(10)

In [22]:
X.shape

(10, 100, 10)

In [23]:
y.shape

(10, 100, 10)

In [24]:
model.fit(X, y, epochs=100, validation_split=0.33)

Train on 6 samples, validate on 4 samples
Epoch 1/100
6/6 [==============================] - 4s 586ms/step - loss: 0.0128 - acc: 0.6383 - val_loss: 0.0162 - val_acc: 0.0000e+00
Epoch 2/100
6/6 [==============================] - 3s 552ms/step - loss: 0.0130 - acc: 0.0000e+00 - val_loss: 0.0165 - val_acc: 0.1750
Epoch 3/100
6/6 [==============================] - 3s 570ms/step - loss: 0.0129 - acc: 0.0000e+00 - val_loss: 0.0163 - val_acc: 0.0850
Epoch 4/100
6/6 [==============================] - 3s 559ms/step - loss: 0.0127 - acc: 0.0983 - val_loss: 0.0163 - val_acc: 0.5250
Epoch 5/100
6/6 [==============================] - 3s 559ms/step - loss: 0.0128 - acc: 0.6383 - val_loss: 0.0164 - val_acc: 0.5300
Epoch 6/100
6/6 [==============================] - 3s 546ms/step - loss: 0.0128 - acc: 0.6383 - val_loss: 0.0163 - val_acc: 0.2025
Epoch 7/100
6/6 [==============================] - 3s 566ms/step - loss: 0.0128 - acc: 0.0800 - val_loss: 0.0161 - val_acc: 0.2025
Epoch 8/100
6/6 [============

Epoch 63/100
6/6 [==============================] - 4s 639ms/step - loss: 0.0126 - acc: 0.0600 - val_loss: 0.0160 - val_acc: 0.0100
Epoch 64/100
6/6 [==============================] - 4s 624ms/step - loss: 0.0126 - acc: 0.0550 - val_loss: 0.0159 - val_acc: 0.1550
Epoch 65/100
6/6 [==============================] - 4s 599ms/step - loss: 0.0127 - acc: 0.1783 - val_loss: 0.0161 - val_acc: 0.0150
Epoch 66/100
6/6 [==============================] - 4s 606ms/step - loss: 0.0128 - acc: 0.0183 - val_loss: 0.0159 - val_acc: 0.1850
Epoch 67/100
6/6 [==============================] - 4s 607ms/step - loss: 0.0127 - acc: 0.1517 - val_loss: 0.0159 - val_acc: 0.2625
Epoch 68/100
6/6 [==============================] - 4s 639ms/step - loss: 0.0127 - acc: 0.2017 - val_loss: 0.0159 - val_acc: 0.0275
Epoch 69/100
6/6 [==============================] - 4s 601ms/step - loss: 0.0127 - acc: 0.0667 - val_loss: 0.0159 - val_acc: 0.0150
Epoch 70/100
6/6 [==============================] - 3s 550ms/step - loss: 0.

In [25]:
model.save('my_model.h5')

In [26]:
temp = model.predict(inp_data, verbose=0)

In [26]:
!ls


baseline.ipynb	     __init__.py    my_model.h5    try_rnn.py
broadcast_vector.py  linear_map.py  try_rnn.ipynb
